In [1]:
import openmc
import openmc.deplete as od
import math
import pandas as pd

In [2]:
fuel = openmc.Material(name="uo2")
fuel.add_element("U", 1, percent_type="ao", enrichment=4.25)
fuel.add_element("O", 2)
fuel.set_density("g/cc", 10.4)

clad = openmc.Material(name="clad")
clad.add_element("Zr", 1)
clad.set_density("g/cc", 6)

water = openmc.Material(name="water")
water.add_element("O", 1)
water.add_element("H", 2)
water.set_density("g/cc", 1.0)
water.add_s_alpha_beta("c_H_in_H2O")
materials = openmc.Materials([fuel])

In [3]:
radii = [0.42, 0.45]
fuel.volume = math.pi * radii[0] ** 2

In [4]:
cyl = openmc.ZCylinder(r=radii[0])
cyl_cell = openmc.Cell(region=-cyl, fill=fuel)
bound_box = openmc.rectangular_prism(1.24, 1.24, boundary_type="reflective")
root_cell = openmc.Cell(fill=openmc.Universe(cells=[cyl_cell]), region=bound_box)
geometry = openmc.Geometry([root_cell])

In [5]:
settings = openmc.Settings()
settings.particles = 1000
settings.inactive = 10
settings.batches = 50

In [6]:
model = openmc.model.Model(geometry, materials, settings)
operator = openmc.deplete.Operator(model, "../openmc/tests/chain_simple.xml")

In [7]:
operator.number.number

array([[5.54176944e+02, 5.54176944e+02, 5.54176944e+02, 5.54176944e+02,
        5.54176944e+02, 5.54176944e+02, 4.94459466e+18, 5.53201699e+20,
        1.22985089e+22, 2.57086309e+22, 9.74726533e+18, 2.53392949e+18]])

In [8]:
nuclides = {}
for nuc,t in fuel.get_nuclide_atom_densities().items():
    nuclides[nuc] = t[1] * 1e24

In [9]:
xs_micro = pd.DataFrame(index=list(nuclides.keys()), columns=['fission', '(n,gamma)'], data=[[0.1, 0.], 
                                                                                            [0.1,0.], 
                                                                                            [0.9,0.], 
                                                                                            [0.4,0.], 
                                                                                            [0.0,0.], 
                                                                                            [0.0,0.]])
xs_micro=xs_micro.append(pd.DataFrame(index=['I135', 'Xe135', 'Xe136', 'Cs135', 'Gd157', 'Gd156'],
                            columns = ['fission', '(n,gamma)'],
                            data=[[0., 0.1],
                                  [0., 0.9],
                                  [0.,0.],
                                  [0.,0.],
                                  [0.,0.1],
                                  [0.,0.1]]))

/tmp/ipykernel_18222/3961105765.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  xs_micro=xs_micro.append(pd.DataFrame(index=['I135', 'Xe135', 'Xe136', 'Cs135', 'Gd157', 'Gd156'],


In [10]:
myop = od.FluxSpectraDepletionOperator(fuel.volume, nuclides, xs_micro, 1e16, '../openmc/tests/chain_simple.xml')

In [13]:
myop.reaction_rates

ReactionRates([[[0., 0.],
                [0., 0.],
                [0., 0.],
                [0., 0.],
                [0., 0.],
                [0., 0.],
                [0., 0.],
                [0., 0.],
                [0., 0.]]])

In [14]:
operator.reaction_rates

ReactionRates([[[0., 0.],
                [0., 0.],
                [0., 0.],
                [0., 0.],
                [0., 0.],
                [0., 0.],
                [0., 0.],
                [0., 0.],
                [0., 0.]]])